L'objectif ici est de convertir chacun de nos models avec Openvino afin que l'inférence soit plus rapide

### Imports

In [3]:
import time
from pathlib import Path
#import cv2
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Markdown

from keras.applications.vgg19 import VGG19
from openvino.runtime import Core



2022-09-13 14:49:29.059357: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-13 14:49:29.059408: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Nous allons directement télécharger le VGG19 de openvino qui est optimisé pour le IR. Notons que ce VGG19 a été construit avec le framework caffe. Cela ne posera pas de problème bien que nous travaillons avec tensorflow car nous alons le convertir en IR qui est indépendante des frameworks

In [4]:
! omz_downloader --h

usage: omz_downloader [-h] [--name PAT[,PAT...]] [--list FILE.LST] [--all]
                      [--print_all] [--precisions PREC[,PREC...]] [-o DIR]
                      [--cache_dir DIR] [--num_attempts N]
                      [--progress_format {text,json}] [-j N]

optional arguments:
  -h, --help            show this help message and exit
  --name PAT[,PAT...]   download only models whose names match at least one of
                        the specified patterns
  --list FILE.LST       download only models whose names match at least one of
                        the patterns in the specified file
  --all                 download all available models
  --print_all           print all available models
  --precisions PREC[,PREC...]
                        download only models with the specified precisions
                        (actual for DLDT networks); specify one or more of:
                        FP32-INT1,FP32-INT8,FP16-INT1,FP32,FP16,FP16-INT8
  -o DIR, --output_dir DIR
  

In [29]:
# download vgg19
! omz_downloader --name vgg19 -o static

################|| Downloading vgg19 ||################

========== Downloading static/public/vgg19/vgg19.prototxt
... 100%, 5 KB, 31828 KB/s, 0 seconds passed

========== Downloading static/public/vgg19/vgg19.caffemodel
... 100%, 561202 KB, 4467 KB/s, 125 seconds passed

========== Replacing text in static/public/vgg19/vgg19.prototxt



La commande suite nous permettra d'obtenir la représentaion IR de notre model. 

In [23]:
! omz_converter --h

usage: omz_converter [-h] [-d DIR] [-o DIR] [--name PAT[,PAT...]]
                     [--list FILE.LST] [--all] [--print_all]
                     [--precisions PREC[,PREC...]] [-p PYTHON] [--mo MO.PY]
                     [--add_mo_arg ARG] [--dry_run] [-j JOBS]

optional arguments:
  -h, --help            show this help message and exit
  -d DIR, --download_dir DIR
                        root of the directory tree with downloaded model files
  -o DIR, --output_dir DIR
                        root of the directory tree to place converted files
                        into
  --name PAT[,PAT...]   convert only models whose names match at least one of
                        the specified patterns
  --list FILE.LST       convert only models whose names match at least one of
                        the patterns in the specified file
  --all                 convert all available models
  --print_all           print all available models
  --precisions PREC[,PREC...]
                      

In [1]:
# convert vgg19 into IR
! omz_converter --name vgg19 -d static -o static_openvino  --precisions FP16

========== Converting vgg19 to IR (FP16)
Conversion command: /home/liganium/anaconda3/bin/python -- /home/liganium/anaconda3/bin/mo --framework=caffe --data_type=FP16 --output_dir=static_openvino/public/vgg19/FP16 --model_name=vgg19 --input=data '--mean_values=data[103.939,116.779,123.68]' --output=prob --input_model=static/public/vgg19/vgg19.caffemodel --input_proto=static/public/vgg19/vgg19.prototxt '--layout=data(NCHW)' '--input_shape=[1, 3, 224, 224]'

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/liganium/Documents/Car Damage toolkit/Approche 2 directement sans reglog/static/public/vgg19/vgg19.caffemodel
	- Path for generated IR: 	/home/liganium/Documents/Car Damage toolkit/Approche 2 directement sans reglog/static_openvino/public/vgg19/FP16
	- IR output name: 	vgg19
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	data
	- Output layers: 	prob
	- Input shapes: 	[1, 3, 224, 224]
	- Source layout: 	Not speci

### Test Inference on the Converted Model

### Load the Model

In [7]:
ir_path = Path("static_openvino/public/vgg19/FP16")

In [8]:
ie = Core()
model = ie.read_model(model=str(ir_path) + "/vgg19.xml", weights=str(ir_path )+ "/vgg19.bin")
compiled_model = ie.compile_model(model=model, device_name="CPU")

### Get Model Information

In [9]:
input_key = compiled_model.input(0)
output_key = compiled_model.output(0)
network_input_shape = input_key.shape

print(input_key)
print(output_key)
print(network_input_shape)

<ConstOutput: names[data] shape{1,3,224,224} type: f32>
<ConstOutput: names[prob] shape{1,1000} type: f32>
{1, 3, 224, 224}


### Load an Image

Load an image, resize it, and convert it to the input shape of the network.

In [10]:
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.vgg19 import  preprocess_input

In [11]:
def prepare_img_224(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x



In [14]:
img_path = "static/7.jpg"
img = prepare_img_224(img_path)
print(img.shape)
img1 = np.reshape(img,(1,3,224,224))
print(img1.shape)

(1, 224, 224, 3)
(1, 3, 224, 224)


### Do Inference

In [15]:
result = compiled_model([img1])[output_key]

print(result.shape)

(1, 1000)
